#***Road Object Detection Model***

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import os
import cv2

In [ ]:
# Define class names for objects on the road
classes = {
    0: "Car",
    1: "Pedestrian",
    2: "Bicycle",
    3: "Motorcycle",
    4: "Bus",
    5: "Truck",
    6: "Traffic Light",
    7: "Stop Sign",
    8: "Crosswalk",
    9: "Road Sign"
}

In [ ]:
# Function to load images and labels for training
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = tf.keras.preprocessing.image.load_img(os.path.join(folder, filename), target_size=(150, 150))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img_array)
        labels.append(int(folder[-1]))  # Extract the class number from the folder name
    return images, labels

In [ ]:
# Load training data
train_data = []
train_labels = []
for i in range(10):  # Assuming there are 10 classes
    folder_name = f"train_class_{i}"
    images, labels = load_images_from_folder(folder_name)
    train_data.extend(images)
    train_labels.extend(labels)

train_data = np.array(train_data)
train_labels = np.array(train_labels)

In [ ]:
# Split data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
val_images = val_images / 255.0

In [ ]:
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 output classes for 10 objects on the road
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

In [ ]:
# Initialize the video capture
cap = cv2.VideoCapture(0)

# Loop to capture video frames
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Preprocess the frame
    resized_frame = cv2.resize(frame, (150, 150))
    normalized_frame = resized_frame / 255.0
    input_frame = np.expand_dims(normalized_frame, axis=0)

    # Predict objects in the frame
    prediction = model.predict(input_frame)
    predicted_class = np.argmax(prediction)

    # Draw bounding box and label on the frame
    label = classes[predicted_class]
    cv2.rectangle(frame, (0, 0), (200, 50), (255, 255, 255), -1)
    cv2.putText(frame, f"Object: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Display the frame
    cv2.imshow('Road Object Detection', frame)

    # Check for 'q' key press to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Above code snippet is using RealTime capturing and identification
#Below code provides offline prediction of the objects from the captures or clicked photo or image.

In [ ]:
# Single prediction,  without OpenCV code
# Load test data

test_data = []
test_labels = []
for i in range(10):  # 10 classes
    folder_name = f"test_class_{i}"
    images, labels = load_images_from_folder(folder_name)
    test_data.extend(images)
    test_labels.extend(labels)

test_data = np.array(test_data)
test_labels = np.array(test_labels)

# Normalize test data
test_data = test_data / 255.0

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_acc)
def predict_single_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    return f"Predicted object: {classes[predicted_class]}"

# Single prediction
image_path = "car_image.jpg"  # Replace with your image path
prediction_result = predict_single_image(image_path)
print(prediction_result)
